In [79]:
from openai import OpenAI
from typing import List, Optional, Dict
import os
import json
import hashlib
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel

class Characters(BaseModel):
    character_name: List[str]

class SpeakerTagging(BaseModel):
    tagged_lines: List[str]

OPENROUTER_API_KEY="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"
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)

with open("app/data/batch_jp.txt", 'r', encoding="utf-8") as file:
    full_text = file.read()

In [80]:
prompt = (
    "Analyze this Japanese text and identify ALL characters."
    "\nFor each character, determine the character name and gender."
    "\nReturn character profiles in JSON format."
    f"\nText: {full_text}"
)

response = client.beta.chat.completions.parse(
    model="google/gemini-2.0-flash-001",
    temperature=0.1,
    messages=[
        {"role": "system", "content": "You are an expert at analyzing Japanese characters."},
        {"role": "user", "content": prompt}
    ],
    response_format=Characters,
)


In [81]:
output = response.choices[0].message.parsed.character_name

In [82]:
def llm_tag_speakers(text: str, profiles: Dict) -> str:
    """Use LLM to tag speakers based on profiles."""
    try:
        prompt = f"""
        Tag speakers for each line in this Japanese text using the character profiles.
        
        CHARACTER PROFILES:
        {json.dumps(profiles, indent=2, ensure_ascii=False)}
        
        TEXT TO TAG:
        {text}
        
        Tag each line with [Speaker]: or [Narration]: format.
        Use character names from profiles.
        """
        
        response = client.beta.chat.completions.parse(
            model="google/gemini-2.0-flash-001",
            temperature=0.1,
            messages=[
                {"role": "system", "content": "You are an expert at identifying speakers in Japanese visual novel text."},
                {"role": "user", "content": prompt}
            ],
            response_format=SpeakerTagging,
        )
        
        result = json.loads(response.choices[0].message.content)
        print(result)
        tagged_lines = []
        
        for line_data in result["tagged_lines"]:
            line = line_data["line"]
            speaker = line_data.get("speaker", "Narration")
            tagged_lines.append(f"[{speaker}]: {line}")
        
        return "\n".join(tagged_lines)
    
    except Exception as e:
            print(f"Error in speaker tagging: {e}")
            # Simple fallback
            lines = text.splitlines()
            tagged_lines = []
            for line in lines:
                if line.strip().startswith('「'):
                    tagged_lines.append(f"[Speaker]: {line}")
                else:
                    tagged_lines.append(f"[Narration]: {line}")
            return "\n".join(tagged_lines)

In [83]:
llm_tag_speakers(full_text, output)

Error in speaker tagging: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=8028, prompt_tokens=7069, total_tokens=15097, completion_tokens_details=None, prompt_tokens_details=None)


'[Narration]: ──空気が動いた。\n[Narration]: それまでボクを包み込んでいた青白い光が遠のいていく。\n[Narration]: やがて見えたのは、薄汚れた天井だ。\n[Narration]: その場所に向けて伸ばしている右腕は、同じく伸ばしている左腕とは少し異なっていた。\n[Narration]: ボクの右腕は、手首から先が剥き出しの機械部品でできている。\n[Narration]: 指は滑らかに動くけれど、そのたびに、こすれる金属音が小さく鳴り響く。\n[Narration]: 続いて、ゆっくりと上半身を起こした。\n[Narration]: それでさっきまで視界に映っていた青白い光の正体が、緩やかなカーブを描いた楕円形の蓋であることがわかる。\n[Narration]: どうやらボクはこのカプセル状の何かの中で眠りについていたようだ。\n[Narration]: ここはどこだろう。\n[Narration]: ボクは誰だろう。\n[Narration]: なぜボクはここにいるんだろう。\n[Narration]: 自問しても答えは見つからない。\n[Narration]: 記憶をさかのぼろうとしても、あるのは青白い光の記憶だけだ。\n[Narration]: ボクは記憶の手がかりを探すため、カプセルから降りて、部屋の中を練り歩くことにした。\n[Narration]: 歩くという行為は知っていても、その行為自体は久しいのか、足下がおぼつかない。\n[Narration]: 倒れてしまわぬように注意を払いながら、ゆっくりと歩みを進めていく。\n[Narration]: 漂う空気は生ぬるく、空気を肺に取り込む度に喉に埃がまとわりついた。\n[Narration]: 天井に開いた穴からは外の光が差し込んでいて、その筋に沿って、空気中の埃が露わになっている。\n[Narration]: おそらくずいぶんの間、この建物に人が入っていないのだろう。\n[Narration]: 足下には、薄い埃が広がっている。\n[Narration]: ここは何をする場所なのだろう。\n[Narration]: 広い部屋には無数の楕円形のカプセルが点在していた。\n[Narration]: その大半は開いている。\n[Narration]: 中身は空だ。\n